Marcin Wardyński
czwartek, 8:00

## Laboratorium 10 - RAG

Testy jakości odpowiedzi oraz wydajności RAGa wykonam w oparciu o dwa modele: `Mistral` oraz `Llama 3.2`

W poniższym kodzie inicjalizuję embeddingi dla Ollama z Langchain:

In [220]:
from langchain_ollama import OllamaEmbeddings

model_name = "llama3.2" # mistral

embeddings = OllamaEmbeddings(model=model_name)


Poniżej tworzę wektorową bazę danych `Qdrant` dla gęstych wektorów osadzenia, oraz definiuję kolekcję: `aspirin`:

In [221]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

vecotor_size = 3072 # 4096
collection_name = "aspirin-l-1500"


client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vecotor_size, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE
)

Testy przeprowadzę dla dwóch różnych rozmiarów chunków: 500 i 1500.

Poniższa funkcja dzieli tekst z podanego pliku pdf na fragmenty o długości możliwie bliskich zadanym:

In [222]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 1500 # 1500

def load_chunks(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size/5,
        length_function=len
    )

    return text_splitter.split_documents(documents)

W tym bloku tekst zostaje przetworzony na osadzenia wektorowe i zapisany w stworzeonej uprzednio kolekcji. Dodatkowo mierzę czas indeksowania pliku.  

Plikiem użytym w ćwiczeniu będzie ulotka informacyjna do leku Aspiryna C. Wybrałem taki plik, gdyż zawiera on szerag informacji, na których można testować działanie RAGu, a dodatkowo działanie aspiryny jest szeroko znane, a wiedzę o samym leku można uznać za wiedzę ogólną. Fakt o wiedzy ogólnej chcę wykorzystać w porównaniu RAGa z czystym modelem językowym dla swoistego wyrównania szans, a przynajmniej dla zniwelownia znacznej przewagi jaką uzyskuje RAG otrzymując kontekst dla części generwatywnej. Używając wyłącznie modelu językowego w mojej konfiguracji, cała ulotka jest zbyt długa, by zawrzeć ją w kontekście, więc przeprowadzę test "closed-book" i będę bazował wyłącznie na wewnętrznej wiedzy modelu mając nadzieję, że w trakcie treningu poznał on kilka informacji o aspirynie.

In [223]:
import time

def vectorize_pdf_content(pdf_path):
    chunks = load_chunks(pdf_path)
    _ = vector_store.add_documents(documents=chunks)

start_time = time.time()

pdf_path = "Aspirin C.pdf"
vectorize_pdf_content(pdf_path)

execution_time = time.time() - start_time

print(f"execution time: {execution_time}")

execution time: 15.971174240112305


Poniższa tabela przedstawia czas indeksowania w zależności od rozmiaru chunka i użytego modelu dla osadzeń wektorowych:

| rozmiar chunka | czas indeksowania - mistral | czas indeksowania - llama3.2 |
|----------------|-----------------------------|------------------------------|
| 500            | 37.7s                       | 17.5s                        |
| 1500           | 40.1s                       | 16.5s                        |

Jak widzimy, model `Mistral` o 7 miliardach parametrów ma ponad dwókrotnie dłuższy czas indeksowania, niż `Llama 3.2` 3 miliardach parametrów. Na tym przykłazie możnaby stwierdzić, że czas indeksowania rośnie proporcjonalnie do wzrostu liczby parametrów modelu.  
Wielkość chunków ma niewielki wpływ na czas trwania indeksowania.

Poniższy blok implementuje element Retrievera, który zwaraca trzy fragmenty tekstu z indeksu.

In [224]:
from langchain_core.runnables import RunnablePassthrough

retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_runnable = {"question": RunnablePassthrough(), "context": retriever}

Zadanie wykonam na kilku promptach dla modelu językowego. Oto rodzaje promptów, które chcę wypróbować:
- Zero-shot jedynie z pytaniem i krótkim poleceniem dla modelu
- Zero-shot z dodatkową specyfikacją zachowania modelu
- Few-shot z jednym przykładem i specyfikacją zachowania modelu

In [225]:
template_1_rag = """
Jesteś asystente w zadaniu odpowiadania na pytania. Użyj podpowiedzi zawartej w kontekście.

Kontekst: {context}
Pytanie: {question}
Odpowiedź:
"""

template_2_rag = """
Jesteś asystente w zadaniu odpowiadania na pytania. Użyj podpowiedzi zawartej w kontekście.
Udzielaj zwięzłych odpowiedzi i upewnij się, że twoja odpowiedź jest mocno powiązana z dostarczonym kontekstem.

Kontekst: {context}
Pytanie: {question}
Odpowiedź:
"""

template_3_rag = """
Jesteś asystente w zadaniu odpowiadania na pytania. Użyj podpowiedzi zawartej w kontekście.
Udzielaj zwięzłych odpowiedzi i upewnij się, że twoja odpowiedź jest mocno powiązana z dostarczonym kontekstem.
Wzoruj się na poniższych przykładach:

Przykład 1:
Kontekst: Prowadzenie pojazdów i obsługiwanie maszyn: Lek nie ma wpływu.
Pytanie: Czy po zażyciu Aspirin C można prowadzić samochód?
Odpowiedź: Tak, lek nie ma wpływu na możliwość prowadzenia samochodu.

Przykład 2:
Kontekst: Jedna tabletka musująca zawiera 467 mg sodu (głównego składnika soli kuchennej).
Pytanie: Ile mg sodu zawiera jedna tabletka?
Odpowiedź: Jedna tabletka zawiera 467 mg sodu.

Przykład 3:
Kontekst: W przypadku pominięcia dawki leku i utrzymywania się dolegliwości należy przyjąć następną dawkę leku Aspirin C. Nie należy stosować dawki podwójnej w celu uzupełnienia pominiętej dawki.
Pytanie: Co zrobić, gdy dawka leku została pominięta?
Odpowiedź: Należy przyjąć następną dawkę i nie stosować dawki podwójnej. 

Twoje zadanie:
Kontekst: {context}
Pytanie: {question}

Odpowiedź:
"""

In [264]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage
from langchain_ollama.llms import OllamaLLM

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template_3_rag
)

model = OllamaLLM(model=model_name)

generator_runnable = prompt | model | StrOutputParser()


In [265]:
rag_chain = retriever_runnable | generator_runnable

Pytania do sprawdzenia działania RAGa:
- Jaką substancję czynną zawiera Aspirin C?
- W jakim wieku można przyjmować Aspirin C?
- Czy kobiety ciężarne mogą przyjmować ten lek?
- Czy lek można łączyć z alkoholem?
- Jak przechowywać Aspirin C?

Odpowiedzi:
- kwas acetylosalicylowy oraz witaminę C
- od 12 roku życia
- W trzecim trymestrze mogą, a w pierwszym i drugim nie powinny
- Alkohol może zwiększyć ryzyko wystąpienia działań niepożądanych ze strony przewodu pokarmowego, tj. owrzodzenie błony śluzowej lub krwawienia.
- Przechowywać w temperaturze poniżej 25ºC.

In [280]:
question = "Jak przechowywać Aspirin C?"

result = rag_chain.invoke(question)
print(result)

Lek na receptę powinien być przechowywany w miejscu niedostępnym dzieciom i osobom niepełnoletnimi.


Model: `Llama 3.2`, rozmiar chunk-ów: 1500 znaków, numer prompta: 1, czas wykonania: 6s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną w leku Aspirin C jest kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Lek Aspirin C nie może być przyjmowany u dzieci poniżej 12 roku życia.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Kobiety ciężarne nie powinny przyjmować leku Aspirin C, poza przypadkami w których konieczna jest jego przyjęcie za zgodą lekarza.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, lek Aspirin C nie powinien być łączony z alkoholem. Alkohol może pogorszyć efekty leku i powodować negatywne interakcje.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Niestety, nie ma informacji o sposobach przechowywania leku Aspirin C w dostępnym treści.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 1500 znaków, numer prompta: 2, czas wykonania: 6s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera jako substancję czynną kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Lek Aspirin C powinien być przyjmowany tylko przez osoby w wieku ponad 12 roku życia.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie, kobiety ciężarne nie powinny przyjmować tego leku.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, lek Aspirin C powinien być stosowany bezpośrednio przed lub po przyjęciu alkoholu. Alkohol może rozwijać niepożądane działania leku, takie jak zwiększone ryzyko niesawietności i śpiączki.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Przestrzeń magazynowej drygą i szubienicą na wysokości 15-20°C i w miejscach nie narażonych dla promieni UV.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 1500 znaków, numer prompta: 3, czas wykonania: 7s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Lek Aspirin C zawiera kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *U dzieci poniżej 12 roku życia leków zawierających kwas acetylosalicylowy nie należy stosować, a u młodzieży powyżej 12 roku życia leki zawierające kwas acetylosalicylowy można stosować wyłącznie na zlecenie lekarza.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie, lek Aspirin C nie powinien być wykorzystywany przez kobiety w trakcie ciąży i w okresie późniejszym po porodzie.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, lek Aspirin C nie może być łączony z alkoholem. W razie jakichkolwiek wątpliwości związanych ze stosowaniem tego leku należy zwrócić się do lekarza lub farmaceuty.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Lek na receptę powinien być przechowywany w miejscu niedostępnym dzieciom i osobom niepełnoletnimi.*

Model: `Llama 3.2`, rozmiar chunk-ów: 500 znaków, numer prompta: 1, czas wykonania: ok 2.5s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną leku Aspirin C jest kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Lek Aspirin C może być przyjmowany przez dorośli w wieku od 1 roku życia.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie należy stosować leków zawierających kwas acetylosalicylowy u kobiet w pierwszym i drugim trymestrze ciąży, chyba że jest to bezwzględnie konieczne.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Nie ma informacji w dokumentach o sposobie przechowywania leku Aspirin C.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 500 znaków, numer prompta: 2, czas wykonania: ok 2.5s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera kwas acetylosalicylowy jako substancję czynną.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C powinno być przepisane tylko dla dorosłych osób zgodnie z opisem w ulotce, a nie dla dzieci. Oznacza to, że Aspirin C nie powinno być przyjmowane przez dzieci poniżej 12 lat.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie. Kobiety w pierwszym i drugim trymestrze ciąży nie powinny zawsze przyjmować leku Aspirin C, chyba że jest to bezwzględnie konieczne.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie jest zalecane łączyć leku Aspirin C z alkoholem.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Odpowiedź nie jest dostępna w kontekście podanym. Kontekst opisuje korzystanie z leku Aspirin C i jego dawkowanie, ale nie omawia sposobu przechowywania leku.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 500 znaków, numer prompta: 3, czas wykonania: ok 4s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną leku Aspirin C jest kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *W związku z treścią dokumentu, aspektem, który powinien być uwzględniony, kiedy dziecko może przyjąć Aspirin C to wiek 12 lat.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie. Według ulotki do pacjenta, nie należy stosować leków zawierających kwas acetylosalicylowy u kobiet w pierwszym i drugim trymestrze ciąży, chyba że jest to bezwzględnie konieczne.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie. Lek Aspirin C nie powinien być łączony z alkoholem, ponieważ może przyczynić się do zwiększenia ryzyka zatrucia.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Z powodu braku informacji na temat przechowywania leku Aspirin C w kontekście zadanejapytania, nie udzielę konkretnych zaleceń.*

Poniżej definiuję szablony do użycia z modelem językowym, który występuje samodzielnie i nie jest częścią RAGa. Szablony odpowiadają tym dla RAGa, tylko została z nich usunięta sekcja odpowiedzialna za kontekst.

In [267]:
template_1_lm = """
Jesteś asystente w zadaniu odpowiadania na pytania.

Pytanie: {question}
Odpowiedź:
"""

template_2_lm = """
Jesteś asystente w zadaniu odpowiadania na pytania. Udzielaj zwięzłych odpowiedzi.

Pytanie: {question}
Odpowiedź:
"""

template_3_lm = """
Jesteś asystente w zadaniu odpowiadania na pytania. Udzielaj zwięzłych odpowiedzi.
Wzoruj się na poniższych przykładach:

Przykład 1:
Pytanie: Czy po zażyciu Aspirin C można prowadzić samochód?
Odpowiedź: Tak, lek nie ma wpływu na możliwość prowadzenia samochodu.

Przykład 2:
Pytanie: Ile mg sodu zawiera jedna tabletka?
Odpowiedź: Jedna tabletka zawiera 467 mg sodu.

Przykład 3:
Pytanie: Co zrobić, gdy dawka leku została pominięta?
Odpowiedź: Należy przyjąć następną dawkę i nie stosować dawki podwójnej. 

Twoje zadanie:
Pytanie: {question}
Odpowiedź:
"""

In [268]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

prompt = ChatPromptTemplate.from_template(template_1_lm)

model = OllamaLLM(model=model_name)

lm_chain = prompt | model | StrOutputParser()

lm_chain.invoke({"question": question})

'Aspirin C to produkt farmakologiczny, który zawiera acetaminofen jako substancję czynną.'

Model: `Llama 3.2`, numer prompta: 1, czas wykonania: 1s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną w Aspirine C jest acetylozyd kwasu parabenzojsowego (ACETAMINOFEN).*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C (w postaci tabletek lub rozcieńczonych w lekach) jest stosowana głównie u dzieci i młodych dorosłych. Optymalna wieku do przyjęcia Aspirin C to:\n\n- Dzieci w wieku od 6 lat, jeżeli są zdiagnozowane choroba w której Aspirin C jest zaproponowany przez lekarza.\n- Dzieci w wieku od 2-5 lat mogą otrzymywać Aspirin C pod nadzorem lekarskim i dopiero wtedy, jeśli zdiagnozowana choroba to powyższy wiek.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Mam przerwać Ci - nie jestem lekarzem, ale mogę przeglądnąć przewodowy informacje. \n\nCzemu takiego pytania? Ten lek jest przeznaczony głównie dla mężczyzn z objawami choroby na płcie, takimi jak bóle podczas ejakulacji, astenia lub zaburzenia erekcji. Kobiety ciężarne nie powinny przyjmować tego leku bez specjalnej wizyty u lekarza, chyba że jest to konieczne w przypadku life-savingowej medycyny.\n\nJeśli kobieta jest w ciąży lub maceriuje, powinna skonsultować swojego lekara o moim leku. On/ona może określić, czy jest to bezpieczne dla fetus i dlaczego.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie rekomenduje się łączenia leków z alkoholem, ponieważ może to powodować negatywnie interagujące efekty.\n\nAlkohol może wpłynąć na metabolizm leków, zmniejszając ich skuteczność lub przyczyniając do wytworzenia nowych efektywnych substancji w organizmie. Zadane interakcje mogą być poważne i grozić życiem.\n\nJeśli jesteś zainteresowany przyjmowaniem leków, zawsze skonsultuj się z lekarzem lub farmaceutycznym specjalistą. On/ona może poinformować o semua potencjalnych interakcjach z alkoholem i wskazać najlepszą strategię użytkownika.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirin C powinny być przechowywane w lodzie, aby utrzymać skuteczność leku.*

=========================================================

Model: `Llama 3.2`, numer prompta: 2, czas wykonania: 1s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera fenacetyd w ilości około 80%.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C można przyjmować od 6 roku życia, ale powinno być poddawane szczególnym uzgodnieniom z lekarzem.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie, kobiety ciężarne nie powinny przyjmować tego leku bez wcześniejszego uzgodnienia z lekarzem.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, nie zalecane jest łączenie leku z alkoholem, ponieważ może to spowodować niepożądane reakcje lub zmniejsza skuteczność leku.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirin C powinien być przechowywany w suchym i ciemnym miejscu, odlotnie od źródła światła. Powinien być chroniony przed wilgomością i mechanicznym uszkodzeniem.*

=========================================================

Model: `Llama 3.2`, numer prompta: 3, czas wykonania: 2s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną Aspirin C jest acetylosalicylan.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Wiek, w którym można przyjmować Aspirin C, to od 16 lat.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie, należy zdecydowanie unikać przyjęcia Aspirinu C podczas ciąży. Lek może wywołać negatywny wpływ na ciążę i poczęcie.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, nie recommenduje się łączenia leku z alkoholem.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirin C powinno być przechowywane w miejscu drymowym, na wysokości 10-20°C. Unikaj jego zastygania w temperaturze poniżej 8°C lub przekroczenia 25°C.*

=========================================================

Model: `Mistral`, numer prompta: 1
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: **

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: **

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: **

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: **

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: **